In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import random

from IPython.display import display

from nltk import word_tokenize, sent_tokenize

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, classification_report
from sklearn.dummy import DummyClassifier
import nltk

In [ ]:
train_url = 'https://media.githubusercontent.com/media/ricadolcvuba/Dataset-TP2/main/df_train_preprocesado.csv'
df_train = pd.read_csv(train_url)
x_train = df_train['review_es']
y_train = df_train['sentimiento']
test_url = 'https://media.githubusercontent.com/media/ricadolcvuba/Dataset-TP2/main/df_test_preprocesado.csv'
df_test = pd.read_csv(test_url)
x_test = df_test['review_es']

In [ ]:
x_train_text = df_train['review_es']
y_train_text = df_train['sentimiento']

x_test_text = df_test['review_es']

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# XGBoost

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, KFold,RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, make_scorer
from xgboost import XGBClassifier
import xgboost as xgb

##Vectorizar

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
my_stop_words = stopwords.words('spanish')

# Instanciamos el CV
#Configurar el vectorizador de palabras para convertir el texto en una representación numérica.
vec = CountVectorizer(stop_words=my_stop_words, ngram_range=(1,2))

# Ajustamos el CV y transformamos los datos
#transformar los datos de entrenamiento en una matriz numérica.
x_train_aux = vec.fit_transform(x_train_text)
#Transformar los datos de prueba en la misma representación numérica que el conjunto de entrenamiento
x_test = vec.transform(x_test_text)

In [ ]:
y_train_binario = y_train.map(lambda x: 1 if x == 'negativo' else 0)

##Modelo

###n=10, cv=10

In [ ]:
#Iteraciones
n=10
#fold
folds = 10
#Kfold estratificado
kfoldcv = KFold(n_splits=folds)

# Definir el espacio de búsqueda de hiperparámetros
param_dist = {
    'learning_rate': np.linspace(0.01, 0.9, 200),
    'max_depth': [3, 4, 5, 6, 7],
    'subsample': np.linspace(0, 1, 30),
    'lambda': np.linspace(0, 10, 20),
    'gamma': np.linspace(0, 9, 20),
    'n_estimators': range(10, 180, 10),
    'eta': np.arange(0.2, 0.7, 0.1)
}

# Configurar la búsqueda de hiperparámetros con RandomizedSearchCV
f1_scorer = make_scorer(f1_score, average='binary', pos_label=1)

modelo_xgb = xgb.XGBClassifier(random_state=0, n_estimators=100)

random_search = RandomizedSearchCV(estimator=modelo_xgb,
                                   param_distributions=param_dist,
                                   scoring=f1_scorer,
                                   n_iter=n,
                                   cv=kfoldcv,
                                   random_state=1,
                                   n_jobs=-1)

random_search.fit(x_train_aux, y_train_binario);
# Obtener el mejor modelo
best_xgb = random_search.best_estimator_

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
#Mejores hiperparametros
print(random_search.best_params_)
#Mejor métrica
print(random_search.best_score_)

{'subsample': 0.5517241379310345, 'n_estimators': 120, 'max_depth': 6, 'learning_rate': 0.4214572864321608, 'lambda': 7.894736842105263, 'gamma': 5.684210526315789, 'eta': 0.6000000000000001}
0.8437072740373918


#### Prediccion y armado de df

In [ ]:
#Mejor estimador
best_xgb=random_search.best_estimator_

#Predicción con mejor estimador
y_pred_xgb=best_xgb.predict(x_test)

In [ ]:
import pandas as pd

# Crear un DataFrame con las predicciones
df_pred_xgb = pd.DataFrame()
df_pred_xgb['ID'] = df_test['ID']  # Asumiendo que df_test['ID'] contiene los IDs correspondientes
df_pred_xgb['sentimiento'] = y_pred_xgb  # Asumiendo que y_pred_xgb contiene las predicciones

In [ ]:
import joblib

In [ ]:
joblib.dump(best_xgb,'/content/drive/MyDrive/modelos/xgboost_n10cv10.joblib')

['/content/drive/MyDrive/modelos/xgboost_n10cv10.joblib']

In [ ]:
df_pred_xgb['sentimiento'] = df_pred_xgb['sentimiento'].apply(lambda x: 'positivo' if x == 0 else 'negativo')

In [ ]:
df_pred_xgb

,ID,sentimiento
0,60000,negativo
1,60001,positivo
2,60002,negativo
3,60003,negativo
4,60004,negativo
...,...,...
8594,68594,positivo
8595,68595,negativo
8596,68596,positivo
8597,68597,negativo


In [ ]:
df_pred_xgb.to_csv('/content/drive/MyDrive/pred/y_pred_xgb_n10cv10.csv', index=False)

In [ ]:
joblib.dump(random_search, '/content/drive/MyDrive/mejores_hiperparametros/random_search_xgb_n10cv10.pkl')

['/content/drive/MyDrive/mejores_hiperparametros/random_search_xgb_n10cv10.pkl']